This notebook is for analysing the tracking data and event data provided by Metrica Sports.

In [1]:
import pandas as pd
import numpy as np

In [ ]:
event_data = pd.read_csv('../sample-data/data/Sample_Game_1/Sample_Game_1_RawEventsData.csv')
event_data.head()


,Team,Type,Subtype,Period,Start Frame,Start Time [s],End Frame,End Time [s],From,To,Start X,Start Y,End X,End Y
0,Away,SET PIECE,KICK OFF,1,1,0.04,0,0.00,Player19,NaN,NaN,NaN,NaN,NaN
1,Away,PASS,NaN,1,1,0.04,3,0.12,Player19,Player21,0.45,0.39,0.55,0.43
2,Away,PASS,NaN,1,3,0.12,17,0.68,Player21,Player15,0.55,0.43,0.58,0.21
3,Away,PASS,NaN,1,45,1.80,61,2.44,Player15,Player19,0.55,0.19,0.45,0.31
4,Away,PASS,NaN,1,77,3.08,96,3.84,Player19,Player21,0.45,0.32,0.49,0.47


In [13]:
event_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1745 entries, 0 to 1744
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Team            1745 non-null   object 
 1   Type            1745 non-null   object 
 2   Subtype         792 non-null    object 
 3   Period          1745 non-null   int64  
 4   Start Frame     1745 non-null   int64  
 5   Start Time [s]  1745 non-null   float64
 6   End Frame       1745 non-null   int64  
 7   End Time [s]    1745 non-null   float64
 8   From            1745 non-null   object 
 9   To              799 non-null    object 
 10  Start X         1664 non-null   float64
 11  Start Y         1664 non-null   float64
 12  End X           1109 non-null   float64
 13  End Y           1109 non-null   float64
dtypes: float64(6), int64(3), object(5)
memory usage: 191.0+ KB


In [14]:
event_data.describe()

,Period,Start Frame,Start Time [s],End Frame,End Time [s],Start X,Start Y,End X,End Y
count,1745.000000,1745.000000,1745.000000,1745.000000,1745.000000,1664.000000,1664.000000,1109.000000,1109.000000
mean,1.488825,70503.382235,2820.135289,70526.925501,2821.077020,0.527139,0.502524,0.524454,0.503201
std,0.500018,41925.812208,1677.032488,41925.545286,1677.021811,0.243625,0.283635,0.242372,0.299590
min,1.000000,1.000000,0.040000,0.000000,0.000000,0.000000,-0.020000,-0.050000,-0.040000
25%,1.000000,32338.000000,1293.520000,32338.000000,1293.520000,0.360000,0.270000,0.350000,0.250000
50%,1.000000,68920.000000,2756.800000,68945.000000,2757.800000,0.530000,0.490000,0.530000,0.490000
75%,2.000000,107787.000000,4311.480000,107802.000000,4312.080000,0.702500,0.752500,0.690000,0.780000
max,2.000000,143622.000000,5744.880000,143630.000000,5745.200000,1.010000,1.010000,1.060000,1.040000


In [4]:
missing_values = event_data.isnull().sum()
print("Missing values in each column:")
print(missing_values)

Missing values in each column:
Team                0
Type                0
Subtype           953
Period              0
Start Frame         0
Start Time [s]      0
End Frame           0
End Time [s]        0
From                0
To                946
Start X            81
Start Y            81
End X             636
End Y             636
dtype: int64


In [ ]:
# check if all the coordinates are within the pitch dimensions ([0,1], length: 105m, width: 68m)
event_coordinates = event_data[['Start X', 'Start Y', 'End X', 'End Y']]
invalid_coordinates = event_coordinates[
    (event_coordinates < 0).any(axis=1) |
    (event_coordinates > 1).any(axis=1)
]
print(f"Number of events with invalid coordinates: {len(invalid_coordinates)}")

Number of events with invalid coordinates: 92


As we can see from the description of evant data, the coordinates are mostly within [0,1], some are not, but slightly bigger or smaller than the limit, it is resonable since players enter or exit the filed which won't affect the outcome.

Analysing Tracking data

In [7]:
tracking_away_data = pd.read_csv('../sample-data/data/Sample_Game_1/Sample_Game_1_RawTrackingData_Away_Team.csv')
tracking_away_data.head()

/var/folders/gv/jv1wmjv920g5v803c075q0qm0000gn/T/ipykernel_70734/2195486640.py:1: DtypeWarning: Columns (0,1,2,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31) have mixed types. Specify dtype option on import or set low_memory=False.
  tracking_away_data = pd.read_csv('../sample-data/data/Sample_Game_1/Sample_Game_1_RawTrackingData_Away_Team.csv')


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Away,Unnamed: 4,Away.1,Unnamed: 6,Away.2,Unnamed: 8,Away.3,...,Away.10,Unnamed: 24,Away.11,Unnamed: 26,Away.12,Unnamed: 28,Away.13,Unnamed: 30,Away.14,Unnamed: 32
0,NaN,NaN,NaN,25,NaN,15,NaN,16,NaN,17,...,24,NaN,26,NaN,27,NaN,28,NaN,NaN,NaN
1,Period,Frame,Time [s],Player25,NaN,Player15,NaN,Player16,NaN,Player17,...,Player24,NaN,Player26,NaN,Player27,NaN,Player28,NaN,Ball,NaN
2,1,1,0.04,0.90509,0.47462,0.58393,0.20794,0.67658,0.4671,0.6731,...,0.37833,0.27383,NaN,NaN,NaN,NaN,NaN,NaN,0.45472,0.38709
3,1,2,0.08,0.90494,0.47462,0.58393,0.20794,0.67658,0.4671,0.6731,...,0.37833,0.27383,NaN,NaN,NaN,NaN,NaN,NaN,0.49645,0.40656
4,1,3,0.12,0.90434,0.47463,0.58393,0.20794,0.67658,0.4671,0.6731,...,0.37833,0.27383,NaN,NaN,NaN,NaN,NaN,NaN,0.53716,0.42556


In [16]:
tracking_away_data.describe()

,Unnamed: 4,Unnamed: 6,Unnamed: 8,Unnamed: 10,Unnamed: 12,Unnamed: 14,Unnamed: 16,Unnamed: 18,Unnamed: 20,Unnamed: 22,Unnamed: 24,Unnamed: 26,Unnamed: 28,Unnamed: 30,Unnamed: 32
count,145006.000000,145006.000000,145006.000000,145006.000000,145006.000000,119381.000000,145006.000000,145006.000000,106869.000000,145006.000000,106599.000000,38408.000000,38138.000000,25626.000000,88251.000000
mean,0.499492,0.518690,0.528137,0.712962,0.437823,0.521667,0.456227,0.556918,0.609951,0.513512,0.509962,0.501591,0.239937,0.460045,0.501049
std,0.068963,0.195041,0.123752,0.196512,0.248207,0.194920,0.185304,0.211405,0.305203,0.199630,0.179647,0.141678,0.130022,0.178857,0.278802
min,0.183170,-0.025180,0.108540,0.008870,-0.024740,0.005520,0.001680,0.002370,-0.050000,-0.023760,0.025280,0.090690,-0.018410,0.067760,-0.039910
25%,0.469180,0.376810,0.452540,0.603450,0.234580,0.392130,0.322723,0.411870,0.334510,0.366520,0.380780,0.419615,0.140852,0.325730,0.253580
50%,0.488480,0.537060,0.524235,0.754005,0.400200,0.515890,0.442990,0.539455,0.691080,0.512080,0.504230,0.489720,0.231540,0.445020,0.499310
75%,0.517440,0.666430,0.595128,0.857920,0.649547,0.643770,0.557517,0.717487,0.882910,0.656697,0.636780,0.556253,0.336800,0.575932,0.732900
max,0.987460,1.050000,1.010750,1.050000,1.013540,1.003740,0.995050,1.040950,1.050000,1.011410,1.003800,0.990100,0.975520,0.985140,1.037820


In [8]:
tra_away_missing_values = tracking_away_data.isnull().sum()
print("Missing values in each column of away team tracking data:")
print(tra_away_missing_values)

Missing values in each column of away team tracking data:
Unnamed: 0          1
Unnamed: 1          1
Unnamed: 2          1
Away                0
Unnamed: 4          2
Away.1              0
Unnamed: 6          2
Away.2              0
Unnamed: 8          2
Away.3              0
Unnamed: 10         2
Away.4              0
Unnamed: 12         2
Away.5          25625
Unnamed: 14     25627
Away.6              0
Unnamed: 16         2
Away.7              0
Unnamed: 18         2
Away.8          38137
Unnamed: 20     38139
Away.9              0
Unnamed: 22         2
Away.10         38407
Unnamed: 24     38409
Away.11        106598
Unnamed: 26    106600
Away.12        106868
Unnamed: 28    106870
Away.13        119380
Unnamed: 30    119382
Away.14         56756
Unnamed: 32     56757
dtype: int64


In [15]:
valid_missing = tra_away_missing_values[tra_away_missing_values > 2]
print("Columns with more than 2 missing values:")
print(valid_missing)

Columns with more than 2 missing values:
Away.5          25625
Unnamed: 14     25627
Away.8          38137
Unnamed: 20     38139
Away.10         38407
Unnamed: 24     38409
Away.11        106598
Unnamed: 26    106600
Away.12        106868
Unnamed: 28    106870
Away.13        119380
Unnamed: 30    119382
Away.14         56756
Unnamed: 32     56757
dtype: int64


In [ ]:
# the missing values in these columns probably because not all players are on the pitch at the same time

In [17]:
tracking_home_data = pd.read_csv('../sample-data/data/Sample_Game_1/Sample_Game_1_RawTrackingData_Home_Team.csv')
tracking_home_data.head()

/var/folders/gv/jv1wmjv920g5v803c075q0qm0000gn/T/ipykernel_70734/1266029072.py:1: DtypeWarning: Columns (0,1,2,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31) have mixed types. Specify dtype option on import or set low_memory=False.
  tracking_home_data = pd.read_csv('../sample-data/data/Sample_Game_1/Sample_Game_1_RawTrackingData_Home_Team.csv')


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Home,Unnamed: 4,Home.1,Unnamed: 6,Home.2,Unnamed: 8,Home.3,...,Home.10,Unnamed: 24,Home.11,Unnamed: 26,Home.12,Unnamed: 28,Home.13,Unnamed: 30,Unnamed: 31,Unnamed: 32
0,NaN,NaN,NaN,11,NaN,1,NaN,2,NaN,3,...,10,NaN,12,NaN,13,NaN,14,NaN,NaN,NaN
1,Period,Frame,Time [s],Player11,NaN,Player1,NaN,Player2,NaN,Player3,...,Player10,NaN,Player12,NaN,Player13,NaN,Player14,NaN,Ball,NaN
2,1,1,0.04,0.00082,0.48238,0.32648,0.65322,0.33701,0.48863,0.30927,...,0.55243,0.43269,NaN,NaN,NaN,NaN,NaN,NaN,0.45472,0.38709
3,1,2,0.08,0.00096,0.48238,0.32648,0.65322,0.33701,0.48863,0.30927,...,0.55243,0.43269,NaN,NaN,NaN,NaN,NaN,NaN,0.49645,0.40656
4,1,3,0.12,0.00114,0.48238,0.32648,0.65322,0.33701,0.48863,0.30927,...,0.55243,0.43269,NaN,NaN,NaN,NaN,NaN,NaN,0.53716,0.42556


In [18]:
tracking_home_data.describe()

,Unnamed: 4,Unnamed: 6,Unnamed: 8,Unnamed: 10,Unnamed: 12,Unnamed: 14,Unnamed: 16,Unnamed: 18,Unnamed: 20,Unnamed: 22,Unnamed: 24,Unnamed: 26,Unnamed: 28,Unnamed: 30,Unnamed: 32
count,145006.000000,46706.000000,145006.000000,145006.000000,145006.000000,145006.000000,110849.000000,145006.000000,145006.000000,145006.000000,121392.000000,98301.000000,34158.000000,23615.000000,88251.000000
mean,0.486946,0.748618,0.536854,0.534354,0.532428,0.559546,0.524862,0.440591,0.522876,0.578405,0.510561,0.537230,0.470764,0.268465,0.501049
std,0.049735,0.155938,0.190341,0.138763,0.260456,0.243916,0.198665,0.271725,0.296344,0.175599,0.241261,0.167934,0.175472,0.160498,0.278802
min,0.110820,0.264610,0.073080,-0.050000,-0.025220,0.011520,-0.007400,-0.008870,-0.050000,0.008940,-0.034680,0.115890,0.042340,0.011800,-0.039910
25%,0.471410,0.628692,0.418010,0.451323,0.301840,0.388653,0.401060,0.226703,0.255800,0.477850,0.336058,0.420200,0.363413,0.122135,0.253580
50%,0.483050,0.756815,0.514300,0.529920,0.543280,0.574945,0.527510,0.378820,0.507080,0.573830,0.499650,0.523490,0.455595,0.274500,0.499310
75%,0.502390,0.874465,0.647005,0.619850,0.757340,0.761327,0.662880,0.624460,0.800930,0.694640,0.684613,0.648790,0.569530,0.387035,0.732900
max,0.820220,1.017230,1.007910,1.004020,1.034940,1.012180,1.003880,1.031870,1.005080,1.050000,1.030120,1.011430,1.002010,0.975910,1.037820


In [19]:
tra_home_missing_values = tracking_home_data.isnull().sum()
print("Missing values in each column of home team tracking data:")
print(tra_home_missing_values)

Missing values in each column of home team tracking data:
Unnamed: 0          1
Unnamed: 1          1
Unnamed: 2          1
Home                0
Unnamed: 4          2
Home.1          98300
Unnamed: 6      98302
Home.2              0
Unnamed: 8          2
Home.3              0
Unnamed: 10         2
Home.4              0
Unnamed: 12         2
Home.5              0
Unnamed: 14         2
Home.6          34157
Unnamed: 16     34159
Home.7              0
Unnamed: 18         2
Home.8              0
Unnamed: 20         2
Home.9              0
Unnamed: 22         2
Home.10         23614
Unnamed: 24     23616
Home.11         46705
Unnamed: 26     46707
Home.12        110848
Unnamed: 28    110850
Home.13        121391
Unnamed: 30    121393
Unnamed: 31     56756
Unnamed: 32     56757
dtype: int64


In [20]:
valid_missing_home = tra_home_missing_values[tra_home_missing_values > 2]
print("Columns with more than 2 missing values in home team tracking data:")
print(valid_missing_home)

Columns with more than 2 missing values in home team tracking data:
Home.1          98300
Unnamed: 6      98302
Home.6          34157
Unnamed: 16     34159
Home.10         23614
Unnamed: 24     23616
Home.11         46705
Unnamed: 26     46707
Home.12        110848
Unnamed: 28    110850
Home.13        121391
Unnamed: 30    121393
Unnamed: 31     56756
Unnamed: 32     56757
dtype: int64


Checking miss frames

In [27]:
frames = tracking_away_data.loc[1: , :]
frames.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Away,Unnamed: 4,Away.1,Unnamed: 6,Away.2,Unnamed: 8,Away.3,...,Away.10,Unnamed: 24,Away.11,Unnamed: 26,Away.12,Unnamed: 28,Away.13,Unnamed: 30,Away.14,Unnamed: 32
1,Period,Frame,Time [s],Player25,NaN,Player15,NaN,Player16,NaN,Player17,...,Player24,NaN,Player26,NaN,Player27,NaN,Player28,NaN,Ball,NaN
2,1,1,0.04,0.90509,0.47462,0.58393,0.20794,0.67658,0.46710,0.6731,...,0.37833,0.27383,NaN,NaN,NaN,NaN,NaN,NaN,0.45472,0.38709
3,1,2,0.08,0.90494,0.47462,0.58393,0.20794,0.67658,0.46710,0.6731,...,0.37833,0.27383,NaN,NaN,NaN,NaN,NaN,NaN,0.49645,0.40656
4,1,3,0.12,0.90434,0.47463,0.58393,0.20794,0.67658,0.46710,0.6731,...,0.37833,0.27383,NaN,NaN,NaN,NaN,NaN,NaN,0.53716,0.42556
5,1,4,0.16,0.90377,0.47463,0.58351,0.20868,0.6764,0.46762,0.67279,...,0.37756,0.27473,NaN,NaN,NaN,NaN,NaN,NaN,0.55346,0.42231


In [28]:
frames.columns = frames.iloc[0]
frames = frames.iloc[1:].reset_index(drop=True)
frames.head()

1,Period,Frame,Time [s],Player25,NaN,Player15,NaN,Player16,NaN,Player17,...,Player24,NaN,Player26,NaN,Player27,NaN,Player28,NaN,Ball,NaN
0,1,1,0.04,0.90509,0.47462,0.58393,0.20794,0.67658,0.46710,0.6731,...,0.37833,0.27383,NaN,NaN,NaN,NaN,NaN,NaN,0.45472,0.38709
1,1,2,0.08,0.90494,0.47462,0.58393,0.20794,0.67658,0.46710,0.6731,...,0.37833,0.27383,NaN,NaN,NaN,NaN,NaN,NaN,0.49645,0.40656
2,1,3,0.12,0.90434,0.47463,0.58393,0.20794,0.67658,0.46710,0.6731,...,0.37833,0.27383,NaN,NaN,NaN,NaN,NaN,NaN,0.53716,0.42556
3,1,4,0.16,0.90377,0.47463,0.58351,0.20868,0.6764,0.46762,0.67279,...,0.37756,0.27473,NaN,NaN,NaN,NaN,NaN,NaN,0.55346,0.42231
4,1,5,0.2,0.90324,0.47464,0.58291,0.21039,0.67599,0.46769,0.67253,...,0.37663,0.27543,NaN,NaN,NaN,NaN,NaN,NaN,0.55512,0.40570


In [30]:
tracking_frames = frames['Frame']
frames_diff = tracking_frames.astype(int).diff()
gaps = frames_diff[frames_diff > 1]
print("Gaps in frame numbers:")
print(len(gaps))

Gaps in frame numbers:
0


In [40]:
# get data of player 15
part1 = tracking_away_data[['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2']]
patr2 = tracking_away_data[['Away.1','Unnamed: 6']]
player_15_data = pd.concat([part1, patr2], axis=1)
player_15_data = player_15_data.iloc[2:, :].reset_index(drop=True)
player_15_data.columns = ['period', 'Frame', 'Time [s]', 'player15_x', 'player15_y']
player_15_data.head()

,period,Frame,Time [s],player15_x,player15_y
0,1,1,0.04,0.58393,0.20794
1,1,2,0.08,0.58393,0.20794
2,1,3,0.12,0.58393,0.20794
3,1,4,0.16,0.58351,0.20868
4,1,5,0.2,0.58291,0.21039


In [41]:
player_15_data.to_csv('../sample-data/data/Sample_Game_1/player_15_data.csv', index=False)